In [ ]:
!pip -q install .
#!pip -q install torch
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

In [ ]:
import numpy as np
from shutil import copyfile
from collections import namedtuple, deque
import random
import time
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from unityagents import UnityEnvironment

from IPython.display import display, clear_output

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
device

In [ ]:
from unityagents import UnityEnvironment
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt

from MultiAgent import Agent

env = Environment("/data/Tennis_Linux_NoVis/Tennis")
    
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# examine the state space 
# states = env_info.vector_observations
# state_size = states.shape[1]
state_size = len(env_info.vector_observations[0])
print('There are {} agents. Each observes a state with length: {}'.format(num_agents, state_size))
print('Action size:\t', action_size)
print('The state for the first agent looks like:', states[0])

In [ ]:
env = Environment("/data/Tennis_Linux_NoVis/Tennis")
# Place the Tennis.app file in the directory and pass "Tennis.app"

In [ ]:
buffer_size = int(1e+5)
batch_size = 128
random_seed = 2

num_episodes = 10000

agent = Agent(num_agents, state_size, action_size, buffer_size, batch_size, random_seed)

def Multi_DDPG(n_episodes=num_episodes):
    print_every=100
    scores_window = deque(maxlen=100)
    scores = []
    max_score = 0
    for i_episode in range(1, n_episodes+1):
        states = env.reset()
        agent.reset()
        score = np.zeros(env.num_agents)
        while True:
            actions = agent.act(states) # send the action to the environment
            # get the next state
            next_states, rewards, dones, _ = env.step(actions)
            agent.step(states, actions, rewards, next_states, dones)
            score = score + rewards
            states = next_states
            
            if np.any(dones):
                break
                
        scores_window.append(score)
        scores.append(score)
        avgscore_ep = np.mean(score)
        avgscore_deque = np.mean(scores_window)
        if i_episode % print_every == 0:
            print(f"Episode: {i_episode}/{num_episodes}\tScore: {avgscore_ep:.2f}\tAverage Score: {avgscore_deque:.2f}")
        if avgscore_deque>=0.5:
            print(f"***Solved! Episodes taken: {i_episode:d} Average Score: {avgscore_deque:.3f}***")
            if avgscore_deque > max_score:
                max_score = avgscore_deque
                torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
                torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')     
                
    return scores

In [ ]:
scores = Multi_DDPG()

# got solved at 4616 episode

In [ ]:
import matplotlib.pyplot as plt


fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()